In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import LFPy
import neuron
import numpy as np
import pandas as pd
import example_network_methods as methods
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import MaxNLocator
from plotting import annotate_subplot
import plotting

In [ ]:
plt.rcParams.update(plotting.rcParams)
golden_ratio = plotting.golden_ratio
figwidth = plotting.figwidth

In [ ]:
np.random.seed(1234)

In [ ]:
neuron.load_mechanisms('mod')

neuron.h.celsius = 34 # does nothing? celsius=34 seems hardcoded in Hay2011 NMODL files

In [ ]:
def get_polycol_from_cell(custom_fun, 
                          custom_fun_args,
                          morphology='BallAndSticks_E.hoc',
                          templatefile='BallAndSticksTemplate.hoc', 
                          templatename='BallAndSticksTemplate',
                          templateargs=None,
                          color='C0',
                          label='E', 
                          pos_args=dict(x=0, y=0, z=0),
                          rotation_args=dict(x=0, y=0, z=0)):
    cellParameters = dict(
        morphology=morphology,
        templatefile=templatefile,
        templatename=templatename,
        custom_fun=custom_fun,
        custom_fun_args=custom_fun_args,
        templateargs=templateargs,
        delete_sections=True,
    )
    cell = LFPy.TemplateCell(**cellParameters)    
    cell.set_pos(**pos_args)
    cell.set_rotation(**rotation_args)

    zips = []
    for x, z in cell.get_idx_polygons(projection=('x', 'z')):
        zips.append(list(zip(x, z)))
    polycol = PolyCollection(zips,
                             edgecolors=color,
                             facecolors=color,
                             linewidths=0.25,
                             label=label)
    return polycol, cell

In [ ]:
def sim_response_ball_n_sticks(custom_fun, custom_fun_args,
                 Vrest=-65, amp=1.,
                 morphology='BallAndSticks_E.hoc', 
                 noise=None, idx=0):
    cellParameters = dict(
        morphology=morphology,
        templatefile='BallAndSticksTemplate.hoc',
        templatename='BallAndSticksTemplate',
        custom_fun=custom_fun,
        custom_fun_args=custom_fun_args,  # [dict(Vrest=Vrest)],
        v_init=Vrest,
        dt=2**-4,
        tstop=1700,
        templateargs=None,
        delete_sections=True,
    )

    cell = LFPy.TemplateCell(**cellParameters)

    stim0 = LFPy.StimIntElectrode(cell=cell, idx=idx, pptype='IClamp',
                                  record_current=True,
                                  amp=-amp, dur=100, delay=1100)
    stim1 = LFPy.StimIntElectrode(cell=cell, idx=idx, pptype='IClamp',
                                  record_current=True,
                                  amp=amp, dur=100, delay=1300)

    stim2 = LFPy.StimIntElectrode(cell=cell, idx=idx, pptype='IClamp',
                                  record_current=True,
                                  amp=0, dur=100, delay=1500)
    if noise is not None:
        noise.play(cell._hoc_stimlist[2]._ref_amp, cell.dt)
    
    cell.simulate()

    # show morphology
    if False:
        if custom_fun == [set_active_hay2011]:
            zips = []
            for x, z in cell.get_idx_polygons(projection=('x', 'z')):
                zips.append(list(zip(x, z)))
            polycol = PolyCollection(zips,
                                     edgecolors='none',
                                     facecolors='gray')

            fig = plt.figure()
            ax = fig.add_subplot(111)
            ax.add_collection(polycol)
            ax.axis(ax.axis('equal'))

    return cell.tvec, cell.somav, stim0.i + stim1.i + stim2.i

    del stim0, stim1, stim2, cell, morphology

In [ ]:
def sim_response_hay2011(templatefile, templatename, custom_fun, custom_fun_args,
                         Vrest=-65, amp=1., noise=None, idx=0):
    cellParameters = dict(
        morphology='L5bPCmodelsEH/morphologies/cell1.asc',
        templatefile=templatefile,
        templatename=templatename,
        templateargs='L5bPCmodelsEH/morphologies/cell1.asc',
        passive=False,
        custom_fun=custom_fun,
        custom_fun_args=custom_fun_args,
        v_init=Vrest,
        dt=2**-4,
        tstop=1700,
        delete_sections=True,
    )

    cell = LFPy.TemplateCell(**cellParameters)
    cell.set_rotation(**dict(x=4.729, y=-3.166))

    stim0 = LFPy.StimIntElectrode(cell=cell, idx=idx, pptype='IClamp',
                                  record_current=True,
                                  amp=-amp, dur=100, delay=1100)
    stim1 = LFPy.StimIntElectrode(cell=cell, idx=idx, pptype='IClamp',
                                  record_current=True,
                                  amp=amp, dur=100, delay=1300)
    stim2 = LFPy.StimIntElectrode(cell=cell, idx=idx, pptype='IClamp',
                                  record_current=True,
                                  amp=0, dur=100, delay=1500)
    if noise is not None:
        noise.play(cell._hoc_stimlist[2]._ref_amp, cell.dt)
    
    cell.simulate()

    return cell.tvec, cell.somav, stim0.i + stim1.i + stim2.i

    del stim0, stim1, stim2, cell, templatefile, custom_fun, custom_fun_args,

In [ ]:
# create some noisy stimuli trace
dt = 2**-4
delay = 1500
dur = 100
sigma = 0.0025
noise = np.r_[np.zeros(int(delay / dt)), (np.random.randn(int(dur / dt)) * sigma).cumsum()]
noise = neuron.h.Vector(noise)

In [ ]:
# E and I colors
colors = ['tab:blue', 'tab:red']

### variable-amplitude step currents and draw amplitude at final time step

In [ ]:
def step_response_ball_n_sticks(custom_fun, custom_fun_args,
                 Vrest=-65, amp=1.,
                 morphology='BallAndSticks_E.hoc', 
                 idx=0):
    cellParameters = dict(
        morphology=morphology,
        templatefile='BallAndSticksTemplate.hoc',
        templatename='BallAndSticksTemplate',
        custom_fun=custom_fun,
        custom_fun_args=custom_fun_args,  # [dict(Vrest=Vrest)],
        v_init=Vrest,
        dt=2**-4,
        tstop=1200,
        templateargs=None,
        delete_sections=True,
    )

    cell = LFPy.TemplateCell(**cellParameters)

    stim = LFPy.StimIntElectrode(cell=cell, idx=idx, pptype='IClamp',
                                 record_current=True,
                                 amp=-amp, dur=1E8, delay=1100)    
    cell.simulate()

    return cell.tvec, cell.somav, stim.i

    del stim0, stim1, stim2, cell, morphology

In [ ]:
def step_response_hay2011(templatefile, templatename, custom_fun, custom_fun_args,
                         Vrest=-65, amp=1., idx=0):
    cellParameters = dict(
        morphology='L5bPCmodelsEH/morphologies/cell1.asc',
        templatefile=templatefile,
        templatename=templatename,
        templateargs='L5bPCmodelsEH/morphologies/cell1.asc',
        passive=False,
        custom_fun=custom_fun,
        custom_fun_args=custom_fun_args,
        v_init=Vrest,
        dt=2**-4,
        tstop=1200,
        delete_sections=True,
    )

    cell = LFPy.TemplateCell(**cellParameters)
    cell.set_rotation(**dict(x=4.729, y=-3.166))

    stim = LFPy.StimIntElectrode(cell=cell, idx=idx, pptype='IClamp',
                                 record_current=True,
                                 amp=-amp, dur=1E8, delay=1100)
    cell.simulate()

    return cell.tvec, cell.somav, stim.i

    del stim0, stim1, stim2, cell, templatefile, custom_fun, custom_fun_args,

In [ ]:
fig = plt.figure(figsize=(figwidth, figwidth))
gs = GridSpec(5, 5, hspace=0.35, wspace=0.45)
axes = []

# number of stimuli amplitudes
n_amps = 10

# morphologies
ax0 = fig.add_subplot(gs[:, :2], aspect='equal')
axes.append(ax0)

# E cell
polycol, _ = get_polycol_from_cell(
    custom_fun=[methods.set_active_hay2011],
    custom_fun_args=[dict(Vrest=-65)],
    morphology='BallAndSticks_E.hoc',
    templatefile='BallAndSticksTemplate.hoc', 
    templatename='BallAndSticksTemplate',
    color=colors[0],
    label='E',
    pos_args=dict(x=0, y=0, z=0))
ax0.add_collection(polycol)

# I cell
polycol, _ = get_polycol_from_cell(
    custom_fun=[methods.set_active_hay2011],
    custom_fun_args=[dict(Vrest=-65)],
    morphology='BallAndSticks_I.hoc',
    templatefile='BallAndSticksTemplate.hoc', 
    templatename='BallAndSticksTemplate',
    color=colors[1],
    label='I',
    pos_args=dict(x=150, y=0, z=0))
ax0.add_collection(polycol)

# Hay cell
polycol, _ = get_polycol_from_cell(
    morphology='L5bPCmodelsEH/morphologies/cell1.asc',
    templatefile=['L5bPCmodelsEH/models/L5PCbiophys3.hoc',
                  'L5bPCmodelsEH/models/L5PCtemplate.hoc'],
    templatename='L5PCtemplate',
    templateargs='L5bPCmodelsEH/morphologies/cell1.asc',
    custom_fun=None,
    custom_fun_args=None,
    color=colors[0],
    label=r'$\mathrm{E}_\mathrm{Hay2011}$',
    pos_args=dict(x=400, y=0, z=0),
    rotation_args=dict(x=4.729, y=-3.166, z=0))
ax0.add_collection(polycol)

# stimulating electrodes
el = plt.Polygon(np.array([[0, 0], [100, 500], [120, 500]]), color='gray')
ax0.add_patch(el)
arrowprops = dict(
    arrowstyle="->",
    connectionstyle="arc3,rad=0.2")
ax0.annotate(('stim'), (110, 500), xytext=(90, 600), arrowprops=arrowprops)

el = plt.Polygon(np.array([[150, 0], [250, 500], [270, 500]]), color='gray')
ax0.add_patch(el)
arrowprops = dict(
    arrowstyle="->",
    connectionstyle="arc3,rad=-0.2")
ax0.annotate((''), (260, 500), xytext=(180, 600), arrowprops=arrowprops)

el = plt.Polygon(np.array([[400, 0], [500, 500], [520, 500]]), color='gray')
ax0.add_patch(el)
arrowprops = dict(
    arrowstyle="->",
    connectionstyle="arc3,rad=-0.2")
ax0.annotate((''), (510, 500), xytext=(190, 600), arrowprops=arrowprops)


ax0.axis(ax0.axis('equal'))

ax0.set_xlabel('$x$ (µm)')
ax0.set_ylabel('$y$ (µm)')
ax0.legend(loc=2)


# stimulus current
ax1 = fig.add_subplot(gs[0, 2])
axes.append(ax1)

# Vm ball-n-sticks E population
ax2 = fig.add_subplot(gs[1, 2], sharex=ax1)
ax3 = fig.add_subplot(gs[2, 2], sharex=ax1, sharey=ax2)
ax4 = fig.add_subplot(gs[3, 2], sharex=ax1, sharey=ax2)
ax5 = fig.add_subplot(gs[4, 2])
for ax in [ax2, ax3, ax4, ax5]:
    axes.append(ax)

# stimulation current amplitudes
amps = np.linspace(-0.2, 0.2, n_amps)

m = 'BallAndSticks_E.hoc'
_, v0, _ = step_response_ball_n_sticks([methods.set_active_hay2011], [dict(Vrest=-65)], -65, .0, m)
Vrest = v0[-1]

# container for "final" Vm value
Vf = np.zeros((3, amps.size))

for ii, amp in enumerate(amps):
    for j, (custom_fun, label, ax) in enumerate(zip(
            [methods.set_active_hay2011,
            methods.set_Ih_linearized_hay2011,
            methods.set_frozen_hay2011,
            # methods.set_pas_hay2011
            ],
            ['active',
            'biophys:lin',
            'biophys:frozen',
            # 'biophys:pas'
            ], 
            [ax2, ax3, ax4])):
        if custom_fun not in [methods.set_active_hay2011]:
            t, v, i = step_response_ball_n_sticks([custom_fun, methods.make_cell_uniform],
                                                [dict(Vrest=Vrest), dict(Vrest=Vrest)],
                                                Vrest, amp, m)
        else:
            t, v, i = step_response_ball_n_sticks([custom_fun],
                                                [dict(Vrest=Vrest)],
                                                Vrest, amp, m)
        Vf[j, ii] = v[-1]
        if j == 0:
            ax1.plot(t[t>=1075], i[t>=1075])        
        ax.plot(t[t>=1075], v[t>=1075], label=label)

        # ax2.legend(loc=2)
        for ax in [ax2, ax3, ax4]:
            ax.set_ylabel(r'$V_\mathrm{m}$ (mV)')
            ax.axis(ax.axis('tight'))


ax4.set_ylim(ymax=-45)

ax1.set_title(r'$\mathrm{E}$')
for j, (title, label, ax) in enumerate(zip(['active', 'quasi-linearized', 'passive-frozen'], 
                                           ['active', 'biophys:lin', 'biophys:frozen'], 
                                           [ax2, ax3, ax4])):
    ax.set_title(title)
    ax5.plot(amps, Vf[j], '-o', label=label, clip_on=False)
# ax5.legend(loc='best')
ax5.set_ylabel(r'$V_\mathrm{m}$ (mV)')
ax5.set_xlabel(r'$I_\mathrm{stim}$ (nA)')

ax1.set_ylabel(r'$I_\mathrm{stim}$ (nA)')
        
for ax in [ax1, ax2, ax3]:
    plt.setp(ax.get_xticklabels(), visible=False)
    # plt.setp(ax2.get_xticklabels(), visible=False)
ax4.set_xlabel('$t$ (ms)')


# stimulus current
ax1 = fig.add_subplot(gs[0, 3])
axes.append(ax1)

# Vm ball-n-sticks I population
ax2 = fig.add_subplot(gs[1, 3], sharex=ax1)
ax3 = fig.add_subplot(gs[2, 3], sharex=ax1, sharey=ax2)
ax4 = fig.add_subplot(gs[3, 3], sharex=ax1, sharey=ax2)
ax5 = fig.add_subplot(gs[4, 3])
for ax in [ax2, ax3, ax4, ax5]:
    axes.append(ax)

# stimulation current amplitudes
amps = np.linspace(-0.1, 0.1, n_amps)

m = 'BallAndSticks_I.hoc'
_, v0, _ = step_response_ball_n_sticks([methods.set_active_hay2011], [dict(Vrest=-65)], -65, .0, m)
Vrest = v0[-1]

# container for "final" Vm value
Vf = np.zeros((3, amps.size))

for ii, amp in enumerate(amps):
    for j, (custom_fun, label, ax) in enumerate(zip(
            [methods.set_active_hay2011,
            methods.set_Ih_linearized_hay2011,
            methods.set_frozen_hay2011,
            # methods.set_pas_hay2011
            ],
            ['active',
            'biophys:lin',
            'biophys:frozen',
            # 'biophys:pas'
            ], 
            [ax2, ax3, ax4])):
        if custom_fun not in [methods.set_active_hay2011]:
            t, v, i = step_response_ball_n_sticks([custom_fun, methods.make_cell_uniform],
                                                [dict(Vrest=Vrest), dict(Vrest=Vrest)],
                                                Vrest, amp, m)
        else:
            t, v, i = step_response_ball_n_sticks([custom_fun],
                                                [dict(Vrest=Vrest)],
                                                Vrest, amp, m)
        Vf[j, ii] = v[-1]
        if j == 0:
            ax1.plot(t[t>=1075], i[t>=1075])        
        ax.plot(t[t>=1075], v[t>=1075], label=label)

        # ax2.legend(loc=2)
        for ax in [ax2, ax3, ax4]:
            # ax.set_ylabel(r'$V_\mathrm{m}$ (mV)')
            ax.axis(ax.axis('tight'))


ax4.set_ylim(ymax=-45)
ax1.set_title(r'$\mathrm{I}$')
for j, (title, label, ax) in enumerate(zip(['active', 'quasi-linearized', 'passive-frozen'], 
                                           ['active', 'biophys:lin', 'biophys:frozen'], 
                                           [ax2, ax3, ax4])):
    ax.set_title(title)
    ax5.plot(amps, Vf[j], '-o', label=label, clip_on=False)
ax5.set_xlabel(r'$I_\mathrm{stim}$ (nA)')
        
for ax in [ax1, ax2, ax3]:
    plt.setp(ax.get_xticklabels(), visible=False)
ax4.set_xlabel('$t$ (ms)')



# stimulus current
ax1 = fig.add_subplot(gs[0, 4])
axes.append(ax1)

# Vm Hay2011 population
ax2 = fig.add_subplot(gs[1, 4], sharex=ax1)
ax3 = fig.add_subplot(gs[2, 4], sharex=ax1, sharey=ax2)
ax4 = fig.add_subplot(gs[3, 4], sharex=ax1, sharey=ax2)
ax5 = fig.add_subplot(gs[4, 4])
for ax in [ax2, ax3, ax4, ax5]:
    axes.append(ax)


templatefiles = [
    ['L5bPCmodelsEH/models/L5PCbiophys3.hoc',
     'L5bPCmodelsEH/models/L5PCtemplate.hoc'],
    ['L5bPCmodelsEH/models/L5PCbiophys3_lin.hoc',
     'L5bPCmodelsEH/models/L5PCtemplate_lin.hoc'],
    ['L5bPCmodelsEH/models/L5PCbiophys3_frozen.hoc',
     'L5bPCmodelsEH/models/L5PCtemplate_frozen.hoc'],
    # ['L5bPCmodelsEH/models/L5PCbiophys3_pas.hoc',
    #  'L5bPCmodelsEH/models/L5PCtemplate_pas.hoc'],
]

templatenames = [
    'L5PCtemplate',
    'L5PCtemplate_lin',
    'L5PCtemplate_frozen',
    # 'L5PCtemplate_pas',
    ]

_, v0, _ = sim_response_hay2011(
    ['L5bPCmodelsEH/models/L5PCbiophys3.hoc',
     'L5bPCmodelsEH/models/L5PCtemplate.hoc'],
    'L5PCtemplate',
     None, None, -77, .0)
Vrest = v0[-1]

custom_funs = [
    None,
    [methods.set_V_R,
     methods.make_cell_uniform],
    [methods.set_V_R,
     methods.make_cell_uniform],
    # [methods.set_V_R,
    #  methods.make_cell_uniform],
]

custom_funs_args = [
    None,
    [dict(Vrest=Vrest)] * 2,
    [dict(Vrest=Vrest)] * 2,
    # [dict(Vrest=Vrest)] * 2,
    ]

# stimulation current amplitudes
amps = np.linspace(-0.4, 0.4, n_amps)

_, v0, _ = step_response_hay2011(
    ['L5bPCmodelsEH/models/L5PCbiophys3.hoc',
     'L5bPCmodelsEH/models/L5PCtemplate.hoc'],
    'L5PCtemplate',
     None, None, -77, .0)
Vrest = v0[-1]

# container for "final" Vm value
Vf = np.zeros((3, amps.size))

for ii, amp in enumerate(amps):
    for j, (templatefile, templatename, custom_fun, custom_fun_args, label, ax) in enumerate(zip(
            templatefiles,
            templatenames,
            custom_funs,
            custom_funs_args,
            ['biophys3', 'lin', 'frozen', 
            # 'pas'
            ],
            [ax2, ax3, ax4]
            )):
        t, v, i = step_response_hay2011(templatefile, templatename, custom_fun, custom_fun_args,
                                    Vrest, amp)
        Vf[j, ii] = v[-1]
        if j == 0:
            ax1.plot(t[t>=1075], i[t>=1075])        
        ax.plot(t[t>=1075], v[t>=1075], label=label)

        for ax in [ax2, ax3, ax4]:
            ax.axis(ax.axis('tight'))

ax4.set_ylim(ymax=-45)

ax1.set_title(r'$\mathrm{E}_\mathrm{Hay2011}$')
for j, (title, label, ax) in enumerate(zip(['active', 'quasi-linearized', 'passive-frozen'], 
                                           ['active', 'biophys:lin', 'biophys:frozen'], 
                                           [ax2, ax3, ax4])):
    ax.set_title(title)
    ax5.plot(amps, Vf[j], '-o', label=label, clip_on=False)
ax5.legend(loc=1, bbox_to_anchor=(0.8, 0.5, 1, 0.5))
ax5.xaxis.set_major_locator(MaxNLocator(3, steps=[1, 2, 4, 5, 10]))
ax5.set_xlabel(r'$I_\mathrm{stim}$ (nA)')
        
for ax in [ax1, ax2, ax3]:
    plt.setp(ax.get_xticklabels(), visible=False)
ax4.set_xlabel('$t$ (ms)')

'''
for ax, title in zip([ax2, ax3, ax4], ['E', 'I', r'$\mathrm{E}_\mathrm{Hay2011}$']):
    ax.set_title(title)

'''
for ax in axes:
    lines=['right', 'top']
    for loc, spine in ax.spines.items():
        if loc in lines:
            spine.set_color('none')

# add panel labels
annotate_subplot(ax0, ncols=5 / 3, nrows=1, letter='A', linear_offset=0.03)
for ax, letter in zip([axes[1], axes[6], axes[11]], 'BCD'): 
    annotate_subplot(ax, ncols=5, nrows=5, letter=letter, linear_offset=0.03)

if not os.path.isdir('figures'):
    os.mkdir('figures')
fig.savefig(os.path.join('figures', 'figure02.pdf'), bbox_inches='tight')